In [2]:
import os
import json
import pandas as pd
import requests

In [62]:
subscription_key = "subscription_key"
cognitive_services_region = "https://eastus.cognitiveservices.azure.com/"
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key':subscription_key
}
text_sentiment_url = cognitive_services_region + '/text/analytics/v3.0-preview.1/sentiment'

In [3]:
df = pd.read_csv('twitter_model.csv')
df.head()

,full_text
0,COVID-19: How it might change us in the future...
1,"As a GP, I fear our Covid-19 lockdown will res..."
2,Coronavirus lockdown complicates Philippines's...
3,So in less then a week once COVID-19 deaths in...
4,"Hmmm, something is off here... https://t.co/qH..."


In [4]:
df_ =df[:1000]
df_.shape

(1000, 1)

In [6]:
documents = {"documents": []}

for idx, row in df_.iterrows():
    documents["documents"].append({
        "id": str(idx + 1),
        "text": row["full_text"]
    })

In [7]:
documents

{'documents': [{'id': '1',
   'text': 'COVID-19: How it might change us in the future for better -\xa0Impact on pollution, water, sanitation, hygiene https://t.co/nbFlOH9WUy'},
  {'id': '2',
   'text': 'As a GP, I fear our Covid-19 lockdown will result in significantly more deaths than we are trying to prevent,https://t.co/ecGbovT0LT'},
  {'id': '3',
   'text': "Coronavirus lockdown complicates Philippines's fight against TB https://t.co/kcWOC71LU7 https://t.co/6u2qREPWlr"},
  {'id': '4',
   'text': "So in less then a week once COVID-19 deaths in the US eclipse 20,000 because of his incompetence, what's he gonna reference then?!?!? https://t.co/2UtBukQG7F"},
  {'id': '5',
   'text': 'Hmmm, something is off here... https://t.co/qHmux6VroD'},
  {'id': '6',
   'text': 'Minister Josephine Teo: Protect our migrant workers from Covid-19 - Sign the Petition! https://t.co/fb2L4pjIh7 via @Change'},
  {'id': '7',
   'text': 'Late Wisdom: 1. Priority is not Testing. COVID testing is only for data

In [63]:
response  = requests.post(text_sentiment_url, headers=headers, json=documents)
sentiments = response.json()

In [64]:
sentiments

{'documents': [{'id': '1',
   'sentiment': 'negative',
   'documentScores': {'positive': 0.15, 'neutral': 0.07, 'negative': 0.78},
   'sentences': [{'sentiment': 'negative',
     'sentenceScores': {'positive': 0.15, 'neutral': 0.07, 'negative': 0.78},
     'offset': 0,
     'length': 131}]},
  {'id': '2',
   'sentiment': 'negative',
   'documentScores': {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0},
   'sentences': [{'sentiment': 'negative',
     'sentenceScores': {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0},
     'offset': 0,
     'length': 132}]},
  {'id': '3',
   'sentiment': 'negative',
   'documentScores': {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0},
   'sentences': [{'sentiment': 'negative',
     'sentenceScores': {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0},
     'offset': 0,
     'length': 111}]},
  {'id': '4',
   'sentiment': 'negative',
   'documentScores': {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0},
   'sentences': [{'sentiment': 'negative',
 

In [9]:
document_df = pd.DataFrame([d["text"] for d in documents["documents"]], index=[d["id"] for d in documents["documents"]], 
                              columns=["text"])

,text
1,COVID-19: How it might change us in the future...
2,"As a GP, I fear our Covid-19 lockdown will res..."
3,Coronavirus lockdown complicates Philippines's...
4,So in less then a week once COVID-19 deaths in...
5,"Hmmm, something is off here... https://t.co/qH..."


In [11]:
document_df.index = document_df.index.rename('id')
document_df.head()

,text
id,
1,COVID-19: How it might change us in the future...
2,"As a GP, I fear our Covid-19 lockdown will res..."
3,Coronavirus lockdown complicates Philippines's...
4,So in less then a week once COVID-19 deaths in...
5,"Hmmm, something is off here... https://t.co/qH..."


In [13]:
sentiment_df = pd.DataFrame([d["sentiment"] for d in sentiments["documents"]], index=[d["id"] for d in sentiments["documents"]],
             columns=["sentiment"])
sentiment_df.head()

,sentiment
1,negative
2,negative
3,negative
4,negative
5,neutral


In [14]:
sentiment_df.index=sentiment_df.index.rename('id')
sentiment_df.head()

,sentiment
id,
1,negative
2,negative
3,negative
4,negative
5,neutral


In [18]:
sentiment_analysis_df = pd.merge(document_df,
                 sentiment_df[ 'sentiment'],
                 left_index=True, right_index=True)

In [20]:
sentiment_analysis_df.head(7)

,text,sentiment
id,,
1,COVID-19: How it might change us in the future...,negative
2,"As a GP, I fear our Covid-19 lockdown will res...",negative
3,Coronavirus lockdown complicates Philippines's...,negative
4,So in less then a week once COVID-19 deaths in...,negative
5,"Hmmm, something is off here... https://t.co/qH...",neutral
6,Minister Josephine Teo: Protect our migrant wo...,neutral
7,Late Wisdom: 1. Priority is not Testing. COVID...,mixed


In [27]:
sentiment_analysis_df.to_csv('text.csv', index='id')